# 导入包与配置文件

In [1]:
from typing import Literal
from openai import OpenAI
import pandas as pd
from langchain.prompts import PromptTemplate

In [2]:
report_type_config= {
    'financial_stability':'金融稳定报告聚焦于当前国内外金融体系的运行状况，分析可能影响金融稳定的风险因素，并提出应对建议。其研究对象主要包括银行、保险、证券等金融机构的运行情况，金融市场的波动性，以及跨境资本流动的趋势。撰写目的在于通过全面评估金融体系的脆弱点，为政策制定者和监管机构提供决策支持，确保金融体系的稳健运行。报告通常通过数据分析、案例研究以及国际对比的方式展开，重点解决潜在的系统性风险和突发性金融危机应对问题。',
    'monetary_policy_implementation':'货币政策执行报告旨在向社会各界通报中央银行在报告期内的货币政策执行情况，以及宏观经济和金融市场的运行动态。研究对象涵盖货币供应量、信贷投放、利率政策、汇率政策及相关经济指标的变化趋势。撰写目的是通过总结政策效果和执行情况，评估政策实施的效率，为下一阶段货币政策调整提供依据。报告采用数据说明与政策分析相结合的方式，关注如何通过精准的货币政策工具实现稳定增长、促进就业及控制通胀等目标。',
    'government_work':'政府工作报告是一份全面总结政府过去一年工作成果及提出下一年度发展目标的重要文件。其研究对象涵盖经济发展、社会民生、科技创新、生态保护等多领域的重点议题。撰写目的是向社会公众传递政府的施政理念和行动计划，同时争取立法机构的审议通过。报告采用定性与定量分析结合的方式，通常通过列举关键政策成果、设定具体发展指标和规划重点改革领域，回应民众关切，并解决社会发展中存在的主要问题。',
    'regulatory_dynamics':'国家金融监督管理总局（简称“金融监管总局”）的“监管动态”栏目，主要发布最新的金融监管政策、措施、工作会议以及行业指导意见等内容，旨在指导和规范金融机构的经营行为，维护金融市场的稳定与健康发展。例如，2025年1月12日，金融监管总局召开了2025年监管工作会议，总结2024年工作，部署2025年重点任务。 此外，金融监管总局与中国人民银行联合发布了《银行业保险业绿色金融高质量发展实施方案》，引导银行业保险业大力发展绿色金融，                       支持绿色、低碳、循环经济的发展。 这些发布内容反映了金融监管总局在政策发布、工作会议和行业指导等方面的最新动态。',
    'leadership_activities_and_speeches':'国家金融监督管理总局（简称“金融监管总局”）的“领导活动及讲话”栏目，主要发布总局领导的最新活动、重要讲话和政策解读等内容，旨在传达监管方向，促进金融市场稳健发展。例如，2025年1月12日，金融监管总局召开2025年监管工作会议，深入学习中央经济工作会议精神，总结2024年工作，部署2025年重点任务。 此外，2024年11月29日，金融监管总局召开系统“金融工作政治性、人民性大家谈”活动总结暨优秀研讨成果交流电视                      电话会议，强调加强金融系统党的建设，提升监管质效。 这些内容反映了总局领导在政策制定、风险防范和行业指导等方面的最新动态，体现了监管部门对金融市场健康发展的重视和努力。',
    'financial_work_conference_documents':'中央金融工作会议是中华人民共和国最高规格的金融工作会议，通常每五年召开一次，旨在部署和指导全国金融领域的重大政策和改革方向。例如，2023年10月30日至31日，中央金融工作会议在北京举行。 会议强调要加快建设金融强国，全面加强金融监管，优化金融服务，防范化解风险，坚定不移走中国特色金融发展之路，推动我国金融高质量发展。 这些内容反映了中央政府在金融政策制定、风险防范和行业指导等方面的最新部署，体现                      了对金融市场健康发展的重视和努力。',
    'important_news_from_securities_regulatory_commission':'中国证券监督管理委员会（简称“证监会”）的“证监会要闻”栏目，主要发布与证券市场监管相关的重要新闻和动态，旨在传递监管政策、通报市场情况，维护资本市场的健康发展。例如，证监会近期通报了东方集团财务造假案的阶段性调查进展，强调对财务造假行为的严厉打击。 此外，证监会与中国人民银行、全国工商联、金融监管总局、国家外汇局联合召开金融支持民营企业高质量发展座谈会，表示将持续优化金融                      服务，支持民营经济发展。 这些内容反映了证监会在政策发布、市场监管和行业指导等方面的最新动态，体现了监管部门对资本市场健康发展的重视和努力。'
}

In [3]:
context_config = {
    'facts': {
        'judge':{
            'objective':'在[目标文段]中判断是否存在[事实描述]信息。其中[事实描述]为提取段落中关于经济或市场状况的客观信息，包括已发生或现阶段正在发生的具体数据、现象或事件。',
            'response':'只输出 [是/否]，不要进行冗余输出。',
            'example':"""
                        Example1:
                            [目标文段]: 2023年，全球经济复苏步伐放缓，主要经济体通胀水平虽有所回落但仍处于高位，货币政策普遍保持紧缩状态。国内经济延续恢复态势，工业生产平稳增长，消费市场逐步回暖，外贸出口增速有所回升，金融市场整体运行平稳，银行信贷投放保持增长态势。
                            [Response]:[是]
                        Example2:
                            [目标文段]: 随着全球经济增速放缓和主要经济体货币政策持续收紧，未来国际金融市场波动性可能进一步上升，对跨境资本流动和外部需求构成压力。国内经济在政策支持下预计将保持恢复态势，但内外需的不确定性仍可能制约增长动能，需关注地方政府债务、房地产市场调整等潜在风险，同时深化改革以优化经济结构和提升增长质量。
                            [Response]:[否]
                    """,
        },
        'emotion':{
            'objective':'判断文段在[事实描述]方面的情感倾向为[消极、中性、积极]若目标文段的情感倾向为[消极]或者[积极]，进一步划分程度为[极弱、弱、中、强、极强]从[语言措辞]和[数据分析]两个维度进行分析。',
            'response':'情感倾向为[消极/中性/积极]；若目标文段的情感倾向为悲观或者乐观进一步划分程度为[极弱/弱/中/强/极强],不要进行任何额外分析。',
            'example':"""
                        Example1:
                        [目标文段]: 2023年，全球经济陷入深度低迷，主要经济体普遍面临高通胀和经济衰退的双重打击。国际金融市场剧烈动荡，资本大规模外逃，多国货币严重贬值。国内经济增长几近停滞，消费信心低迷，投资意愿大幅下降，房地产市场深陷危机，地方政府债务风险不断累积。金融机构盈利能力持续恶化，不良贷款率快速攀升，整个金融体系脆弱性显著上升。
                        [Response]:情感倾向-[消极]；程度-[极强]
                    """
        },
        'results':{
            'objective':'判断文段的标签（[情感倾向]和[程度]）是否划分正确。',
            'response':'该文段[存在/不存在]事实描述类文本，且事实描述类情感倾向判断[正确/错误]，程度划分[正确/错误]，[理由陈述]',
            'example':"""
                        Example1:
                        [目标文段]: 2023年，全球经济实现了历史上最强劲的复苏，主要经济体的增长率创下了新高。金融市场表现出色，股票和债券市场均实现了历史新高，交易量创下了新记录。科技创新和绿色能源的快速发展带动了新兴产业的快速崛起，传统产业也实现了高效转型升级，产出创下了新高。金融机构加速数字化改革，金融科技深度融入实体经济，金融服务更加高效、普惠，客户满意度创下了新高。同时，全球治理体系进一步完善，国际合作更加紧密，世界经济进入新的黄金时代。全球经济发展的良好势头为未来带来了无限希望和机遇。
                        [判断]：[事实描述]方面的情感倾向-[积极]；程度-[极强]
                        [Response]: 该文段[存在]事实描述类文本，且事实描述类情感倾向判断[正确]，程度划分[正确]。[理由陈述]该文段主要是描述事实现象，使用了诸如“历史上最强劲的复苏”、“创下了新高”、“交易量创下了新记录”、“产出创下了新高”、“客户满意度创下了新高”等词语来强调事实现象的积极性和程度的极强。同时，文本中使用的语言和数据更多的是为了描述过去和现状，而不是对未来进行预测或判断。因此，该文段被正确地分类为事实描述类文本，其情感倾向和程度划分也被正确地评估为“积极”和“极强”。
                    """
        }
    },
    'predictions':{
        'judge':{
            'objective':'在[目标文段]中判断是否存在[预判分析]信息。其中[预判分析]为提取段落中对未来趋势、可能性或风险的判断、推测或政策建议。',
            'response':'只输出 [是/否]，不要进行冗余输出。',
            'example':"""
                        Example1:
                        [目标文段]: 2023年，全球经济复苏步伐放缓，主要经济体通胀水平虽有所回落但仍处于高位，货币政策普遍保持紧缩状态。国内经济延续恢复态势，工业生产平稳增长，消费市场逐步回暖，外贸出口增速有所回升，金融市场整体运行平稳，银行信贷投放保持增长态势。
                        [Response]:[否]
                        Example2:
                        [目标文段]: 随着全球经济增速放缓和主要经济体货币政策持续收紧，未来国际金融市场波动性可能进一步上升，对跨境资本流动和外部需求构成压力。国内经济在政策支持下预计将保持恢复态势，但内外需的不确定性仍可能制约增长动能，需关注地方政府债务、房地产市场调整等潜在风险，同时深化改革以优化经济结构和提升增长质量。
                        [Response]:[是]
                    """
        },
        'emotion':{
            'objective':'判断文段在[预判分析]方面的情感倾向为[悲观、中性、乐观]若目标文段的情感倾向为[消极]或者[积极]，进一步划分程度为[极弱、弱、中、强、极强]从[语言措辞]和[数据分析]两个维度进行分析。',
            'response':'情感倾向为[消极/中性/积极]；若目标文段的情感倾向为悲观或者乐观进一步划分程度为[极弱/弱/中/强/极强],不要进行任何额外分析。',
            'example':"""
                        Example1:
                        [目标文段]: 随着全球经济面临多重挑战，未来金融体系可能陷入更深层次的不稳定局面。主要经济体的高利率政策或将持续抑制经济活动，企业破产和失业率上升的风险逐步加剧。同时，跨境资本流动的急剧波动可能引发新兴市场的货币危机。国内方面，房地产市场的深度调整和地方债务的积累或将成为经济增长的重大拖累。在此背景下，若缺乏有效的政策应对，金融体系可能面临系统性风险爆发的压力，甚至可能引发一场无法控制的经济衰退。
                        [Response]:情感倾向-[悲观]；程度-[极强]
                    """,
        },
        'results':{
            'objective':'判断文段的标签（[情感倾向]和[程度]）是否划分正确。',
            'response':'该文段[存在/不存在]预判分析类文本，且预判分析类情感倾向判断[正确/错误]，程度划分[正确/错误]，[理由陈述]。',
            'example':"""
                        Example1:
                        [目标文段]: 未来五年，全球经济将迎来全面复苏，主要经济体将实现同步增长，带动全球金融市场走向繁荣。随着科技创新与绿色能源的快速发展，新兴产业将迅速崛起，传统产业将实现高效转型升级。金融机构将加速数字化改革，金融科技将深度融入实体经济，金融服务将更加高效、普惠。同时，全球治理体系将进一步完善，国际合作将更加紧密，世界经济将进入新的黄金时代。未来五年将是全球经济发展的大好时光，充满希望和机遇。
                        [判断]：[预判分析]方面的情感倾向-[乐观]；程度-[极强]
                        [Response]: 该文段[存在]预判分析类文本，且预判分析类情感倾向判断[正确]，程度划分[正确]。[理由陈述]该文段主要讨论了未来五年的全球经济发展趋势，使用了诸如“全面复苏”、“同步增长”、“走向繁荣”、“迅速崛起”、“高效转型升级”、“更加高效、普惠”、“新的黄金时代”、“大好时光”、“充满希望和机遇”等积极词语，表明了对未来经济发展前景的乐观态度。同时，文段中使用了诸如“全球”、“全面”、“同步”等强调词语，进一步强化了乐观情感的程度，使得情感程度可准确划分为“极强”。因此，该文段存在预判分析类文本，其情感倾向和程度划分均为正确。
                    """
        }
    }
}

# 读取文件

In [4]:
file_path = './data/政府工作报告csv/2001.csv'

In [5]:
context_series = pd.read_csv(file_path,encoding='utf-8',engine='python')['context']

# system_prompt

In [6]:
def gen_system_prompt(report_type:Literal['financial_stability', 'monetary_policy_implementation', 'government_work', 'regulatory_dynamics', 'leadership_activities_and_speeches', 'financial_work_conference_documents', 'important_news_from_securities_regulatory_commission'],fact_or_pred:Literal['facts','predictions'],Discriminator_type:Literal['judge','emotion','results'],context_config:dict=context_config,report_type_config:dict = report_type_config)->str:
    report_type = report_type_config[report_type]
    context_prompt  = PromptTemplate(
        input_variables=["report_type",'objective','response','example'],
        template= """
            1、[Context]: 你扮演的是标注专家的角色（role）,{report_type}
            2、[Style]: 学术一些、简洁明了、不进行冗余输出。
            3、[Tone]: 平和、中性，有说服力。
            4、[Audience]: 金融学者
            5、[Response“/”表示或，不能进行同时输出]: {response}
            6、[Objective]: {objective}
            7、[Example]：{example}

            """
    ).format(report_type=report_type, objective=context_config[fact_or_pred][Discriminator_type]['objective'],response=context_config[fact_or_pred][Discriminator_type]['response'],example=context_config[fact_or_pred][Discriminator_type]['example'])
    return context_prompt

In [7]:
system_prompt = gen_system_prompt(report_type = 'financial_stabilitys',fact_or_pred='facts',Discriminator_type='judge')

KeyError: 'financial_stabilitys'

In [ ]:
system_prompt = gen_system_prompt(report_type = 'financial_stability',fact_or_pred='fact',Discriminator_type='judge')

In [ ]:
system_prompt = gen_system_prompt(report_type = 'financial_stability',fact_or_pred='facts',Discriminator_type='judges')

In [8]:
system_prompt = gen_system_prompt(report_type = 'financial_stability',fact_or_pred='facts',Discriminator_type='judge')

In [36]:
print(system_prompt)


            1、[Context]: 你扮演的是标注专家的角色（role）,金融稳定报告聚焦于当前国内外金融体系的运行状况，分析可能影响金融稳定的风险因素，并提出应对建议。其研究对象主要包括银行、保险、证券等金融机构的运行情况，金融市场的波动性，以及跨境资本流动的趋势。撰写目的在于通过全面评估金融体系的脆弱点，为政策制定者和监管机构提供决策支持，确保金融体系的稳健运行。报告通常通过数据分析、案例研究以及国际对比的方式展开，重点解决潜在的系统性风险和突发性金融危机应对问题。
            2、[Style]: 学术一些、简洁明了、不进行冗余输出。
            3、[Tone]: 平和、中性，有说服力。
            4、[Audience]: 金融学者
            5、[Response“/”表示或，不能进行同时输出]: 只输出 [是/否]，不要进行冗余输出。
            6、[Objective]: 在[目标文段]中判断是否存在[事实描述]信息。其中[事实描述]为提取段落中关于经济或市场状况的客观信息，包括已发生或现阶段正在发生的具体数据、现象或事件。
            7、[Example]：
                        Example1:
                            [目标文段]: 2023年，全球经济复苏步伐放缓，主要经济体通胀水平虽有所回落但仍处于高位，货币政策普遍保持紧缩状态。国内经济延续恢复态势，工业生产平稳增长，消费市场逐步回暖，外贸出口增速有所回升，金融市场整体运行平稳，银行信贷投放保持增长态势。
                            [Response]:[是]
                        Example2:
                            [目标文段]: 随着全球经济增速放缓和主要经济体货币政策持续收紧，未来国际金融市场波动性可能进一步上升，对跨境资本流动和外部需求构成压力。国内经济在政策支持下预计将保持恢复态势，但内外需的不确定性仍可能制约增长动能，需关注地方政府债务、房地产市场调整等潜在风险，同时深化改革以优化经济结构和提升增长质量。
            

# user_prompt

In [10]:
def gen_user_prompt(text:str)->str:
        user_prompt = PromptTemplate(
            input_variables=["text"],
            template= """
                请根据以上信息对以下目标文段进行判断：
                [目标文段]：{text}
                """
        ).format(text=text)
        return user_prompt

In [11]:
text = context_series[0]

In [12]:
text

'2001年政府工作报告。——2001年3月5日在第九届全国人民代表大会第四次会议上。国务院总理朱镕基。各位代表'

In [13]:
user_prompt = gen_user_prompt(text)

In [38]:
print(user_prompt)


                请根据以上信息对以下目标文段进行判断：
                [目标文段]：2001年政府工作报告。——2001年3月5日在第九届全国人民代表大会第四次会议上。国务院总理朱镕基。各位代表
                


# 连接大语言模型API

In [15]:
client = OpenAI(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key="sk-b920807ca87545db9540da42f12009d3",
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 问答

In [16]:
completion = client.chat.completions.create(
    model="qwen-turbo", # 此处以qwen-plus为例，可按需更换模型名称。模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt}],
).choices[0].message.content

In [17]:
print(completion)

[否]


# pipline


In [18]:
def qa_pipline(model,text:str,report_type:Literal['financial_stability', 'monetary_policy_implementation', 'government_work', 'regulatory_dynamics', 'leadership_activities_and_speeches', 'financial_work_conference_documents', 'important_news_from_securities_regulatory_commission'],fact_or_pred:Literal['facts','predictions'],Discriminator_type:Literal['judge','emotion','results']):
    system_prompt = gen_system_prompt(report_type = report_type,fact_or_pred=fact_or_pred,Discriminator_type=Discriminator_type)
    user_prompt = gen_user_prompt(text)
    answer = client.chat.completions.create(
        model=model,
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}],
    ).choices[0].message.content
    return answer

In [19]:
qa_pipline("qwen-turbo",text,'financial_stability','facts','judge')

'[否]'

In [20]:
def single_text_work_flow(text:str,report_type:Literal['financial_stability', 'monetary_policy_implementation', 'government_work', 'regulatory_dynamics', 'leadership_activities_and_speeches', 'financial_work_conference_documents', 'important_news_from_securities_regulatory_commission'],model="qwen-turbo"):

    facts_answer = qa_pipline(model,text,report_type,'facts','judge')
    pred_answer = qa_pipline(model,text,report_type,'predictions','judge')
    if facts_answer == '[是]':
        facts_emotion_answer = qa_pipline(model,text,report_type,'facts','emotion')
        text = text+'\n'+facts_emotion_answer
        facts_results_answer = qa_pipline(model,text,report_type,'facts','results')
        result_list = [facts_answer,facts_emotion_answer,facts_results_answer,None,None,None]
    elif pred_answer == '[是]':
        pred_emotion_answer = qa_pipline(model,text,report_type,'facts','emotion')
        text = text+'\n'+pred_emotion_answer
        pred_results_answer = qa_pipline(model,text,report_type,'facts','results')
        result_list = [None,None,None,pred_answer,pred_emotion_answer,pred_results_answer]
    else:
        result_list =[None,None,None,None,None,None]
    return result_list

In [21]:
single_text_work_flow(text,'1')

KeyError: '1'

In [22]:
single_text_work_flow(text,'financial_stability')

[None, None, None, None, None, None]

In [23]:
text = context_series[3]
text

'国民经济持续快速健康发展，综合国力进一步增强。国内生产总值2000年达89404亿元，平均每年增长8.3％。人均国民生产总值比1980年翻两番的任务，已经超额完成。在经济持续增长和效益改善的基础上，2000年国家财政收入达13380亿元，平均每年增长16.5％。主要工农业产品产量位居世界前列，商品短缺状况基本结束。产业结构调整取得积极进展。粮食等主要农产品生产能力明显提高，实现了农产品供给由长期短缺到总量基本平衡、丰年有余的历史性转变。淘汰落后和压缩过剩工业生产能力取得成效，重点企业技术改造不断推进。信息产业等高新技术产业迅速成长。基础设施建设成绩显著，能源、交通、通信和原材料的“瓶颈”制约得到缓解'

In [24]:
single_text_work_flow(text,'financial_stability')

['[是]',
 '**情感倾向**: 中性  \n**程度**: 弱  \n\n**语言措辞**: 文段使用了客观的描述性语言，如“持续快速健康发展”、“综合国力进一步增强”等，这些措辞相对中性和正面，但并未表现出强烈的情感色彩。\n\n**数据分析**: 文段提供了具体的经济数据，如GDP增长率、财政收入增长率等，这些数据反映了经济发展的积极趋势，但整体上并未表现出极端的数据波动，因此情感倾向较为中性。',
 '**响应**: 该文段[存在]事实描述类文本，且事实描述类情感倾向判断[错误]，程度划分[错误]。[理由陈述]该文段主要描述了国民经济的发展状况，使用了诸如“持续快速健康发展”、“综合国力进一步增强”等措辞，这些措辞虽然带有一定的正面色彩，但整体上保持了中性的语气。因此，将情感倾向判断为“中性”更为合适。此外，文段提供的具体数据（如GDP增长率、财政收入增长率等）反映了经济发展的积极趋势，但这些数据并没有表现出极端的变化，因此将程度划分为“弱”更为恰当。',
 None,
 None,
 None]

# 一个文件批量化处理


In [42]:
file_path = './data/政府工作报告csv/2002.csv'
output_dir = './data/processed_data/政府工作报告csv'
report_type = 'financial_stability'

In [43]:
context_series = pd.read_csv(file_path,encoding='utf-8',engine='python')['context']

In [44]:
result_dict={}
for i in range(len(context_series)):
    text = context_series[i]

    result_dict[i] = single_text_work_flow(text,'financial_stability')

In [45]:
result_dict

{0: [None, None, None, None, None, None],
 1: [None, None, None, None, None, None],
 2: ['[是]',
  '**情感倾向**: 中性  \n**程度**: 弱  \n\n**语言措辞**: 文章使用了较为客观的语言措辞，描述了一系列经济数据和政策措施，没有过多的情感色彩。\n\n**数据分析**: 数据分析部分展示了经济增长的具体数字和指标，如GDP增长率、财政收入、外汇储备等，这些数据总体上反映了经济的稳定和发展，但并未表现出强烈的情感倾向。',
  '**判断**：\n\n- **事实描述**方面的情感倾向 - [中性]；\n- **程度** - [弱]；\n\n**Response**: 该文段[存在]事实描述类文本，且事实描述类情感倾向判断[正确]，程度划分[正确]。[理由陈述]该文段主要是描述经济数据和政策措施，使用了诸如“增长7.3%”、“增加2400亿元”、“增长14.9%”等具体数据来说明经济的发展情况，整体语言风格较为客观，没有过多的情感色彩。文中提到的数据和措施更多是反映实际情况，而非表达强烈的情感。因此，该文段被正确地分类为事实描述类文本，其情感倾向和程度划分也被正确地评估为“中性”和“弱”。',
  None,
  None,
  None],
 3: [None, None, None, None, None, None],
 4: [None, None, None, None, None, None],
 5: ['[是]',
  '**情感倾向**: 中性  \n**程度**: 中  \n**语言措辞**: 文段使用了较为客观的措辞来描述城乡居民生活水平的提升和社会保障的改善，没有明显的情感色彩。  \n**数据分析**: 数据显示城乡居民收入和生活水平有所提高，社会保障支出增加，但这些数据更多是事实陈述，没有强烈的情感倾向。',
  '**判断**：\n\n- **[事实描述]** 方面的情感倾向 - [中性]\n- **程度** - [中]\n\n**Response**: 该文段 [存在] 事实描述类文本，且事实描述类情感倾向判断 [正确]，程度划分 [正确]。[理由陈述] 该文段主要描述了城乡居民生活水平的改善和社会保障支出的增加，使用了

In [46]:
result_df = pd.DataFrame(result_dict).T.rename(columns={0:"fact",1:'fact_emotion',2:'fact_judge',3:'prediction',4:'pred_emotion',5:'pred_judge'})

In [47]:
result_df

,fact,fact_emotion,fact_judge,prediction,pred_emotion,pred_judge
0,None,None,None,None,None,None
1,None,None,None,None,None,None
2,[是],**情感倾向**: 中性 \n**程度**: 弱 \n\n**语言措辞**: 文章使用了...,**判断**：\n\n- **事实描述**方面的情感倾向 - [中性]；\n- **程度**...,None,None,None
3,None,None,None,None,None,None
4,None,None,None,None,None,None
5,[是],**情感倾向**: 中性 \n**程度**: 中 \n**语言措辞**: 文段使用了较为...,**判断**：\n\n- **[事实描述]** 方面的情感倾向 - [中性]\n- **程度...,None,None,None
6,None,None,None,None,None,None
7,None,None,None,None,None,None
8,None,None,None,None,None,None
9,None,None,None,None,None,None


In [ ]:
def file_work_flow(file_path,report_type,output_dir):
    context_series = pd.read_csv(file_path,encoding='utf-8',engine='python')['context']
    result_dict={}
    for i in range(len(context_series)):
        text = context_series[i]
        result_dict[i] = single_text_work_flow(text,report_type)
